In [3]:
#Modules
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import re
import numbers

In [ ]:
#Functions
def read_fasta(name):
    fasta_seqs = SeqIO.parse(open(name + '.fa'),'fasta')
    data = []
    for fasta in fasta_seqs:
        data.append([fasta.id, str(fasta.seq).strip().upper()])
            
    return data

In [ ]:
#Data
grna_list = pd.read_csv('grna_list_to_rank.csv')  
gene_list = pd.read_csv('sd108_genetable.csv')  
name_conversion = pd.read_csv('protein_mrna_gene_transcript_ids.csv', index_col=0)
in_fasta_1 = 'sd108'
seqs_df = pd.DataFrame(read_fasta(in_fasta_1), columns = ['name', 'sequence'])

In [4]:
#Gene Density
gene_den = []
genes_in_vicinity = []

for i in range(np.shape(grna_list)[0]):
    count = 0
    grna_sca = grna_list['scaffold'][i]
    grna_loc = grna_list['loc'][i]
    grna_strand = grna_list['strand'][i]
    curr_gene_vicinity = []
    
    curr_goi = gene_list.loc[gene_list['chrom'] == grna_sca].reset_index(drop=True)
    if np.shape(curr_goi)[0] > 0:
        scaffold_len = len(seqs_df['sequence'][int(grna_sca.split('_')[1])-1])
        if grna_strand == '+':
            r_limit = grna_loc + 10000
            l_limit = grna_loc - 10000   
        else:
            r_limit = scaffold_len - grna_loc + 10000
            l_limit = scaffold_len - grna_loc - 10000
    
        if r_limit > scaffold_len:
            r_limit = scaffold_len
        if l_limit < 0:
            l_limit = 0
    
        for j in range(np.shape(curr_goi)[0]):
            if curr_goi['txStart'][j] > l_limit and curr_goi['txEnd'][j] < r_limit:
                curr_gene_vicinity.append(curr_goi['name'][j])
                count = count + 1
           
    genes_in_vicinity.append(', '.join(curr_gene_vicinity))
    gene_den.append(count)
    
grna_list['gene_density'] = gene_den
grna_list['genes_in_vicinity'] = genes_in_vicinity

In [5]:
#Intergenic Region Size [up is after grna and low is before grna irrespective of the strand]
grna_to_select = []
intg_size = []
gene_distance_in_prox = []
T_P = []
genes_3prime_5prime = []

for i in range(np.shape(grna_list)[0]):
    one_gene_flag = 0
    grna_sca = grna_list['scaffold'][i]
    grna_loc = grna_list['loc'][i]
    grna_str = grna_list['strand'][i]
    if grna_str == '+':
        curr_grna_loc = grna_loc
    else:
        curr_grna_loc = len(seqs_df['sequence'][int(grna_list['scaffold'][i].split('_')[1])-1]) - grna_loc

    curr_goi = gene_list.loc[gene_list['chrom'] == grna_sca].reset_index(drop=True)
    
    if np.shape(curr_goi)[0] == 0:
        grna_to_select.append('No')
        intg_size.append('-')
        gene_distance_in_prox.append(['-','-'])
        T_P.append(['-','-'])
        genes_3prime_5prime.append(['-','-'])
        
    else:
        low_limit = 0
        up_limit = len(seqs_df['sequence'][int(grna_sca.split('_')[1])-1])
        low_index = 0
        up_index = 0

        for j in range(np.shape(curr_goi)[0]):
            if curr_goi['txStart'][j] < curr_grna_loc and curr_goi['txEnd'][j] < curr_grna_loc: #txend sufficient
                if curr_goi['txEnd'][j] > low_limit:
                    low_limit = curr_goi['txEnd'][j]
                    low_index = j

            if curr_goi['txStart'][j] > curr_grna_loc and curr_goi['txEnd'][j] > curr_grna_loc: #txstart sufficient
                if curr_goi['txStart'][j] < up_limit:
                    up_limit = curr_goi['txStart'][j]
                    up_index = j
            
        if up_limit == len(seqs_df['sequence'][int(grna_sca.split('_')[1])-1]) or low_limit == 0:
            one_gene_flag = 1
            
            intg_size.append(-1)
            if low_limit == 0:
                if grna_str == '+':
                    gene_distance_in_prox.append([np.abs(up_limit-curr_grna_loc), '-'])
                    genes_3prime_5prime.append([curr_goi['name'][up_index],'-'])
                else:
                    gene_distance_in_prox.append(['-', np.abs(up_limit-curr_grna_loc)])
                    genes_3prime_5prime.append(['-',curr_goi['name'][up_index]])
                    
                if curr_goi['strand'][up_index] == '+':
                    if grna_str == '+':
                        T_P.append(['P','-'])
                    else:
                        T_P.append(['-','P'])
                        
                    if up_limit - curr_grna_loc > 425:
                        grna_to_select.append('Yes')
                    else:
                        grna_to_select.append('No')
                else:
                    if grna_str == '+':
                        T_P.append(['T','-'])
                    else:
                        T_P.append(['-','T'])
                        
                    if up_limit - curr_grna_loc > 125:
                        grna_to_select.append('Yes')
                    else:
                        grna_to_select.append('No')
                        
            else:
                if grna_str == '+':
                    gene_distance_in_prox.append(['-', np.abs(low_limit-curr_grna_loc)])
                    genes_3prime_5prime.append(['-',curr_goi['name'][low_index]])
                else:
                    gene_distance_in_prox.append([np.abs(low_limit-curr_grna_loc), '-'])
                    genes_3prime_5prime.append([curr_goi['name'][low_index],'-'])
                    
                if curr_goi['strand'][low_index] == '+':
                    if grna_str == '+':
                        T_P.append(['-','T'])
                    else:
                        T_P.append(['T','-'])
                        
                    if curr_grna_loc - low_limit > 125:
                        grna_to_select.append('Yes')
                    else:
                        grna_to_select.append('No')
                        
                else:
                    if grna_str == '+':
                        T_P.append(['-','P'])
                    else:
                        T_P.append(['P','-']) 
                            
                    if curr_grna_loc - low_limit > 425:
                        grna_to_select.append('Yes')
                    else:
                        grna_to_select.append('No')
                        
        else:
            intg_size.append(up_limit - low_limit)
            
            if grna_str == '+':
                gene_distance_in_prox.append([np.abs(up_limit-curr_grna_loc), np.abs(low_limit-curr_grna_loc)])
                genes_3prime_5prime.append([curr_goi['name'][up_index],curr_goi['name'][low_index]])
            else:
                gene_distance_in_prox.append([np.abs(low_limit-curr_grna_loc), np.abs(up_limit-curr_grna_loc)])
                genes_3prime_5prime.append([curr_goi['name'][low_index],curr_goi['name'][up_index]])
                
            if curr_goi['strand'][low_index] == '+' and curr_goi['strand'][up_index] == '+':
                if grna_str == '+':
                    T_P.append(['P','T'])
                else:
                    T_P.append(['T','P'])
                    
                if up_limit - curr_grna_loc > 425 and curr_grna_loc - low_limit > 125:
                    grna_to_select.append('Yes')
                else:
                    grna_to_select.append('No')

            elif curr_goi['strand'][low_index] == '+' and curr_goi['strand'][up_index] == '-':
                T_P.append(['T','T'])
                if up_limit - curr_grna_loc > 125 and curr_grna_loc - low_limit > 125:
                    grna_to_select.append('Yes')
                else:
                    grna_to_select.append('No')

            elif curr_goi['strand'][low_index] == '-' and curr_goi['strand'][up_index] == '+':
                T_P.append(['P','P'])
                if up_limit - curr_grna_loc > 425 and curr_grna_loc - low_limit > 425:
                    grna_to_select.append('Yes')
                else:
                    grna_to_select.append('No')

            else:
                if grna_str == '+':
                    T_P.append(['T','P'])
                else:
                    T_P.append(['P','T'])
                    
                if up_limit - curr_grna_loc > 125 and curr_grna_loc - low_limit > 425:
                    grna_to_select.append('Yes')
                else:
                    grna_to_select.append('No')

grna_list['intergenic_region_size'] = intg_size
grna_list['3"_gene_dist'] = [i[0] for i in gene_distance_in_prox]
grna_list['3"_T_or_P'] = [i[0] for i in T_P]
grna_list['3"_gene'] = [i[0] for i in genes_3prime_5prime]
grna_list['5"_gene_dist'] = [i[1] for i in gene_distance_in_prox]
grna_list['5"_T_or_P'] = [i[1] for i in T_P]
grna_list['5"_gene'] = [i[1] for i in genes_3prime_5prime]
grna_list['grna_to_select'] = grna_to_select

grna_list_f = grna_list[grna_list.grna_to_select == 'Yes'].reset_index(drop=True)

In [6]:
#Gene Essentiality Information from Saccharomyces Homology Comparison
homology_data = pd.read_csv('blast.csv', delimiter='\t', header=None)  

essential_protein_id = []
for i in range(np.shape(homology_data)[0]):
    essential_protein_id.append(int(homology_data[1][i].split('|')[2]))
    
unique_essential_protein_id = list(set(essential_protein_id))
print(len(unique_essential_protein_id))

mrna_name = []
for i in range(len(unique_essential_protein_id)):
    mrna_name.append(name_conversion['mrna'][name_conversion[name_conversion['protein ID'] == unique_essential_protein_id[i]].index.tolist()[0]])

gene_ess_count = []
for i in range(np.shape(grna_list_f)[0]):
    curr_gene_ess_count = 0
    curr_gene_vicinity_2 = [grna_list_f['5"_gene'][i],grna_list_f['3"_gene'][i]]
    for j in range(len(curr_gene_vicinity_2)):
        if curr_gene_vicinity_2[j] in mrna_name:
            curr_gene_ess_count = curr_gene_ess_count + 1
            
    if curr_gene_ess_count > 0:
        gene_ess_count.append(curr_gene_ess_count)
    else:
        gene_ess_count.append('NA')
        
grna_list_f['adjacent_gene_ess'] = gene_ess_count

939


In [7]:
#Output
grna_list_filtered_f = grna_list_f.loc[(grna_list_f['gene_density'] > 4) & (grna_list_f['intergenic_region_size'] < 3000) & (grna_list_f['intergenic_region_size'] > 0)].reset_index(drop=True)
grna_list_filtered_f['intergenic_region_size'] = grna_list_filtered_f['intergenic_region_size'].replace([-1], '-')
#grna_list_filtered_f.to_csv('grna_list_with_gene_vicinity_info_filtered.csv',index=False)

grna_list_f['intergenic_region_size'] = grna_list_f['intergenic_region_size'].replace([-1], '-')
del grna_list_f['grna_to_select']
#grna_list_f.to_csv('grna_list_with_gene_vicinity_info_total.csv',index=False)

In [10]:
#Transcriptomics data
expr_data = pd.read_excel('Top10_based_on_TPM.xlsx')

mrna_name = []

for i in range(np.shape(expr_data)[0]):
    mrna_name.append(name_conversion['mrna'][name_conversion[name_conversion['protein ID'] == int(expr_data['Conserved stable gene'][i].split('Issorie2|')[1])].index.tolist()[0]])

expr_data['mrna'] = mrna_name

avg_expr_vic_2 = []

for i in range(np.shape(grna_list_f)[0]):
    if grna_list_f['gene_density'][i] > 0:
        curr_expr_vic_2 = []
            
        curr_gene_vicinity_2 = [grna_list_f['5"_gene'][i],grna_list_f['3"_gene'][i]]
        for j in range(len(curr_gene_vicinity_2)):
            if curr_gene_vicinity_2[j] in mrna_name:
                curr_index = mrna_name.index(curr_gene_vicinity_2[j])
                curr_expr_vic_2.append(np.median([expr_data['IO126_LOD_16h'][curr_index], expr_data['IO126_LOD_16h.1'][curr_index], expr_data['IO126_LOD_16h.2'][curr_index]]))
        
        if len(curr_expr_vic_2) > 0:
            avg_expr_vic_2.append(np.mean(curr_expr_vic_2))
        else:
            avg_expr_vic_2.append('NA')
        
    else:
        avg_expr_vic_2.append('NA')
        
grna_list_f['avg_expr_vic_2'] = avg_expr_vic_2

In [13]:
#On-target activity (RULE SET 2, Doench 2016)
on_target_file = 'on_target_activity.txt'
with open(on_target_file) as file:
    lines = file.readlines()
    on_target_activity = [np.float64(line.rstrip().split(': ')[1]) for line in lines]
    
grna_list_f['on_target_activity'] = on_target_activity

In [15]:
#Output
grna_list_f.to_csv('output.csv',index=False)